In [1]:
import requests
from bs4 import BeautifulSoup
import time
import random
import math
import sqlite3
import pandas as pd
import numpy as np
import urllib
from lxml import etree
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import sqlite3
import pickle
import math
import numpy as np

In [2]:
#initial request
url = 'https://www.basketball-reference.com/'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

In [3]:
#find links for individual page
urls = []
for i in soup.find_all('tr', class_="full_table"):
    urls.append(url + i.find('a', href=True)['href'])
urls

['https://www.basketball-reference.com//teams/MIL/2020.html',
 'https://www.basketball-reference.com//teams/TOR/2020.html',
 'https://www.basketball-reference.com//teams/BOS/2020.html',
 'https://www.basketball-reference.com//teams/MIA/2020.html',
 'https://www.basketball-reference.com//teams/IND/2020.html',
 'https://www.basketball-reference.com//teams/PHI/2020.html',
 'https://www.basketball-reference.com//teams/BRK/2020.html',
 'https://www.basketball-reference.com//teams/ORL/2020.html',
 'https://www.basketball-reference.com//teams/WAS/2020.html',
 'https://www.basketball-reference.com//teams/CHO/2020.html',
 'https://www.basketball-reference.com//teams/CHI/2020.html',
 'https://www.basketball-reference.com//teams/NYK/2020.html',
 'https://www.basketball-reference.com//teams/DET/2020.html',
 'https://www.basketball-reference.com//teams/ATL/2020.html',
 'https://www.basketball-reference.com//teams/CLE/2020.html',
 'https://www.basketball-reference.com//teams/LAL/2020.html',
 'https:

In [4]:
with open('listofurls.pkl', 'wb') as f:
    pickle.dump(urls, f)

In [5]:
with open('listofurls.pkl', 'rb') as f:
    urls = pickle.load(f)

In [28]:
option = webdriver.ChromeOptions()
option.add_argument(" — incognito")
driver = webdriver.Chrome(executable_path='/Users/Anubhav/Desktop/Flatiron School/QCstats/chromedriver', options=option)

In [29]:
all_individuals = pd.DataFrame()
all_teamstats = pd.DataFrame()
for url in range(len(urls)):
    #go to team site
    driver.get(urls[url])
    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.ID, "all_roster")))

    #create dataframes for each table
    df0 = pd.read_html(driver.find_element_by_css_selector('table[id="per_game"]').get_attribute('outerHTML'))[0]
    df1 = pd.read_html(driver.find_element_by_css_selector('table[id="totals"]').get_attribute('outerHTML'))[0]
    df2 = pd.read_html(driver.find_element_by_css_selector('table[id="per_minute"]').get_attribute('outerHTML'))[0]
    df3 = pd.read_html(driver.find_element_by_css_selector('table[id="per_poss"]').get_attribute('outerHTML'))[0]
    df4 = pd.read_html(driver.find_element_by_css_selector('table[id="advanced"]').get_attribute('outerHTML'))[0]
    df5 = pd.read_html(driver.find_element_by_css_selector('table[id="shooting"]').get_attribute('outerHTML'))[0]
    df6 = pd.read_html(driver.find_element_by_css_selector('table[id="pbp"]').get_attribute('outerHTML'))[0] 
    df7 = pd.read_html(driver.find_element_by_css_selector('table[id="salaries2"]').get_attribute('outerHTML'))[0]
    df8 = pd.read_html(driver.find_element_by_css_selector('table[id="roster"]').get_attribute('outerHTML'))[0]

    #change column names of all player tables
    df0.columns = ['Rk', 'Name', 'Age', 'G', 'GS', 'MP/G', 'FGMade/G', 'FGA/G', 'FGPerc',
                   'ThreeMade/G', 'ThreeA/G', 'ThreePerc','TwoMade/G', 'TwoA/G',
                   'TwoPerc', 'eFGPerc', 'FTMade/G', 'FTA/G', 'FTPerc', 'ORB/G',
                   'DRB/G', 'TRB/G','AST/G', 'STL/G', 'BLK/G', 'TOV/G', 'PF/G', 'PTS/G']
    df1.columns = ['Rk', 'Name', 'Age', 'G', 'GS', 'MPTotal', 'FGMadeTotal',
                   'FGATotal', 'FGPerc', 'ThreeMadeTotal', 'ThreeATotal', 'ThreePerc',
                   'TwoMadeTotal', 'TwoATotal', 'TwoPerc', 'eFGPerc', 'FTMadeTotal',
                   'FTATotal', 'FTPerc', 'ORBTotal', 'DRBTotal', 'TRBTotal',
                   'ASTTotal', 'STLTotal', 'BLKTotal', 'TOVTotal', 'PFTotal', 'PTSTotal']
    df1.drop(df1.tail(1).index, inplace=True)
    df2.columns = ['Rk', 'Name', 'Age', 'G', 'GS', 'MP/THREESIX', 'FGMade/THREESIX',
                   'FGA/THREESIX', 'FGPerc', 'ThreeMade/THREESIX', 'ThreeA/THREESIX',
                   'ThreePerc','TwoMade/THREESIX', 'TwoA/THREESIX', 'TwoPerc',
                   'FTMade/THREESIX', 'FTA/THREESIX', 'FTPerc', 'ORB/THREESIX', 'DRB/THREESIX',
                   'TRB/THREESIX', 'AST/THREESIX', 'STL/THREESIX', 'BLK/THREESIX',
                   'TOV/THREESIX', 'PF/THREESIX', 'PTS/THREESIX']
    df3.columns = ['Rk', 'Name', 'Age', 'G', 'GS', 'MP/Hund', 'FGMade/Hund', 'FGA/Hund',
                   'FGPerc', 'ThreeMade/Hund', 'ThreeA/Hund', 'ThreePerc',
                   'TwoMade/Hund', 'TwoA/Hund', 'TwoPerc', 'FTMade/Hund', 'FTA/Hund',
                   'FTPerc', 'ORB/Hund', 'DRB/Hund', 'TRB/Hund', 'AST/Hund', 'STL/Hund',
                   'BLK/Hund', 'TOV/Hund', 'PF/Hund', 'PTS/Hund', 'Blank', 'ORTG', 'DRTG']
    df4.columns = ['Rk', 'Name', 'Age', 'G', 'MPTotal', 'PER', 'TSPerc', 'PercThreeA',
                   'PercFTA/FG', 'PercORB', 'PercDRB', 'PercTRB', 'PercAST', 'PercSTL',
                   'PercBLK', 'PercTOV', 'USG', 'Blank1', 'OWS', 'DWS', 'WS', 'WS/48',
                   'Blank2', 'OBPM', 'DBPM', 'BPM', 'VORP']
    df5.columns = ['Rk', 'Name', 'Age', 'G', 'MPTotal', 'FGPerc', 'AvgDis',
                   'PercTwoA', 'PercZeroThreefeetA','PercThreefeetTenA', 'PercTenSixTA',
                   'PercSixTThreeA', 'PercThreeA', 'TwoPerc', 'ZeroThreePerc',
                   'ThreefeetTenPerc', 'TenSixTPerc', 'SixTThreePerc', 'ThreePerc',
                   'PercTwoAstd', 'PercFGADunks', 'MadeDunks', 'PercThreeAstd',
                   'Perc3PACorner', 'ThreePercCorner', 'HeavesA', 'MadeHeaves']
    df6.columns = ['Rk', 'Name', 'Age', 'G', 'MP', 'PercasPG', 'PercasSG',
                   'PercasSF', 'PercasPF', 'PercasC', 'PMP100OnCourt', 'NPMP100',
                   'TOVBadPass', 'TOVLostBall', 'ShootFoulsC', 'OFoulsC',
                   'ShootFoulsD', 'OFoulsD', 'PGA', 'FAAnd1', 'FABlkd']
    df7.columns = ['Rk', 'Name', 'Salary']
    df7.Salary = df7.Salary.str.replace(',', '').str[1:]
    df7.drop_duplicates(subset='Name', keep='first',inplace=True)
    df8.columns = ['No', 'Name', 'Pos', 'Ht', 'Wt', 'Birth Date', 'Country', 'Exp', 'College']
    df8.Ht = (pd.to_numeric(df8.Ht.str.split('-').str[0])*12) + pd.to_numeric(df8.Ht.str.split('-').str[1])
    df8.drop(['No', 'Pos', 'Wt', 'Birth Date', 'Country', 'College'], axis = 1, inplace = True)
    df8.drop_duplicates(subset='Name', keep='first',inplace=True)
    
    for table in range(0,9):
        exec(f"df{table}.Name = df{table}.Name.str.split().str[0] + ' ' + df{table}.Name.str.split().str[1]")

    #drop uncessary columns from each dataframe
    for table in range(1,9):
        checklist = []
        for x in range(0,table):
            exec(f"checklist.extend(list(df{x}.columns))")
            checklist.remove('Name')
        exec(f"df{table}.drop([col for col in df{table}.columns if col in checklist], axis = 1, inplace = True)")
        exec(f"blank = [col for col in df{table} if col.startswith('Blank')]")
        if not blank:
            pass
        else:
            exec(f"df{table}.drop(blank, axis = 1, inplace = True)")

    #full outer join all dataframes
    for x in range(0,9):
        exec(f"df{x}.set_index('Name', inplace = True)")
        
    namesnotindf7index = [x for x in df7.index if x not in df0.index]
    namesnotindf8index = [x for x in df8.index if x not in df0.index]
    df7.drop(namesnotindf7index, axis=0, inplace=True)
    df8.drop(namesnotindf8index, axis=0, inplace=True)
    
    result = pd.concat([df0, df1, df2, df3, df4, df5, df6, df7, df8], axis=1, sort = True)
    result.drop('Rk', axis = 1, inplace = True)
    result.reset_index(inplace = True)
    result.rename(columns = {'index': 'Name'}, inplace = True)

    #add TeamName column
    for idx, x in enumerate(result.Name):
        result.loc[idx, 'Team'] = urls[url][44:47]
    
    #add rows to pre-existing dataframe
    all_individuals = pd.concat([all_individuals, result], ignore_index=True)
    
    
    teamstats = pd.read_html(driver.find_element_by_css_selector('table[id="team_and_opponent"]').get_attribute('outerHTML'))[0]

    if 'FG.1' in teamstats.columns:
        ending = '.1'
    else:
        ending = '%'
        
    yerp = teamstats[teamstats.index == 0].drop('Unnamed: 0', axis=1)
    yerp.reset_index(drop=True, inplace=True)
    yerp1 = teamstats[teamstats.index == 1].drop(['Unnamed: 0', 'G', 'FG' + ending, '3P' + ending, '2P' + ending, 'FT' + ending], axis=1)
    yerp1.reset_index(drop=True, inplace=True)
    yerp2 = teamstats[teamstats.index == 4].drop('Unnamed: 0', axis=1)
    yerp2.reset_index(drop=True, inplace=True)
    yerp3 = teamstats[teamstats.index == 5].drop(['Unnamed: 0', 'G', 'FG' + ending, '3P' + ending, '2P' + ending, 'FT' + ending], axis=1)
    yerp3.reset_index(drop=True, inplace=True)
    yerp4 = teamstats[teamstats.index == 3].drop(['Unnamed: 0', 'G'], axis=1)
    yerp4.reset_index(drop=True, inplace=True)
    yerp5 = teamstats[teamstats.index == 4].drop(['Unnamed: 0', 'G'], axis=1)
    yerp5.reset_index(drop=True, inplace=True)

    yerp.columns = ['G', 'MPTotal', 'FGMadeTotal', 'FGATotal', 'FGPerc', 'ThreeMadeTotal', 'ThreeATotal',
     'ThreePerc', 'TwoMadeTotal', 'TwoATotal', 'TwoPerc', 'FTMadeTotal', 'FTATotal',
     'FTPerc', 'ORBTotal', 'DRBTotal', 'TRBTotal', 'ASTTotal', 'STLTotal','BLKTotal',
     'TOVTotal', 'PFTotal', 'PTSTotal']

    yerp1.columns = ['MP/G', 'FGMade/G', 'FGA/G', 'ThreeMade/G', 'ThreeA/G',
                     'TwoMade/G', 'TwoA/G', 'FTMade/G', 'FTA/G', 'ORB/G',
                     'DRB/G', 'TRB/G', 'AST/G', 'STL/G','BLK/G', 'TOV/G', 'PF/G', 'PTS/G']

    yerp2.columns = ['OppG', 'OppMPTotal', 'OppFGMadeTotal', 'OppFGATotal', 'OppFGPerc',
                     'OppThreeMadeTotal', 'OppThreeATotal', 'OppThreePerc', 'OppTwoMadeTotal',
                     'OppTwoATotal', 'OppTwoPerc', 'OppFTMadeTotal', 'OppFTATotal', 'OppFTPerc',
                     'OppORBTotal', 'OppDRBTotal', 'OppTRBTotal', 'OppASTTotal', 'OppSTLTotal',
                     'OppBLKTotal', 'OppTOVTotal', 'OppPFTotal', 'OppPTSTotal']

    yerp3.columns = ['OppMP/G', 'OppFGMade/G', 'OppFGA/G', 'OppThreeMade/G', 'OppThreeA/G',
                     'OppTwoMade/G', 'OppTwoA/G', 'OppFTMade/G', 'OppFTA/G', 'OppORB/G',
                     'OppDRB/G', 'OppTRB/G', 'OppAST/G', 'OppSTL/G','OppBLK/G', 'OppTOV/G',
                     'OppPF/G', 'OppPTS/G']

    yerp4.columns = ['MPDiff', 'FGMadeDiff', 'FGADiff', 'FGDiff', 'ThreeMadeDiff', 'ThreeADiff',
                     'ThreePercDiff', 'TwoMadeDiff', 'TwoADiff', 'TwoDiff', 'FTMadeDiff', 'FTADiff',
                     'FTPercDiff', 'ORBDiff', 'DRBDiff', 'TRBDiff', 'ASTDiff', 'STLDiff','BLKDiff',
                     'TOVDiff', 'PFDiff', 'PTSDiff']

    yerp5.columns = ['OppMPDiff', 'OppFGMadeDiff', 'OppFGADiff', 'OppFGDiff', 'OppThreeMadeDiff', 'OppThreeADiff',
                     'OppThreePercDiff', 'OppTwoMadeDiff', 'OppTwoADiff', 'OppTwoDiff', 'OppFTMadeDiff', 'OppFTADiff',
                     'OppFTPercDiff', 'OppORBDiff', 'OppDRBDiff', 'OppTRBDiff', 'OppASTDiff', 'OppSTLDiff','OppBLKDiff',
                     'OppTOVDiff', 'OppPFDiff', 'OppPTSDiff']

    misc = pd.read_html(driver.find_element_by_css_selector('table[id="team_misc"]').get_attribute('outerHTML'))[0]

    misc.columns = ['Label', 'W', 'L', 'PW', 'PL', 'MOV', 'SOS',
                    'SRS', 'ORtg', 'DRtg', 'Pace', 'FTr', '3PAr',
                    'OeFGPerc', 'OTOVPerc', 'ORBPerc', 'OFT/FGA', 'DeFGPerc',
                    'DTOVPerc', 'DRBPerc', 'DFT/FGA', 'Arena', 'Attendance']

    yerp6 = misc[misc.index == 0].drop(['Label', 'Arena', 'Attendance'], axis=1)
    yerp6.reset_index(drop=True, inplace=True)

    teamstatsrow = pd.concat([yerp, yerp1, yerp2, yerp3, yerp4, yerp5, yerp6], axis=1, sort=True)
    teamstatsrow.loc[0, 'Team'] = urls[url][44:47]
    
    all_teamstats = pd.concat([all_teamstats, teamstatsrow], ignore_index=True)
    all_teamstats.reset_index(drop=True, inplace=True)

In [39]:
all_individuals.to_pickle("individualstats.pkl")
all_teamstats.to_pickle("1920teamstats.pkl")

In [10]:
all_individuals = pd.read_pickle("individualstats.pkl")

In [38]:
all_individuals

,Name,Age,G,GS,MP/G,FGMade/G,FGA/G,FGPerc,ThreeMade/G,ThreeA/G,...,OFoulsC,ShootFoulsD,OFoulsD,PGA,FAAnd1,FABlkd,Salary,Ht,Exp,Team
0,Brook Lopez,31,61,60,26.6,4.0,9.4,0.427,1.4,4.7,...,18,61,3,240,13,20,12093024,84.0,11,MIL
1,D.J. Wilson,23,31,0,9.0,1.3,3.3,0.396,0.5,1.8,...,1,5,3,54,1,3,2961120,82.0,2,MIL
2,Donte DiVincenzo,23,59,22,23.1,3.5,7.7,0.462,1.3,3.7,...,8,30,17,317,2,23,2905800,76.0,1,MIL
3,Dragan Bender,22,7,0,13.0,1.4,3.0,0.476,0.6,1.3,...,0,2,0,21,1,4,1678854,NaN,NaN,MIL
4,Eric Bledsoe,30,56,56,27.2,5.7,11.8,0.482,1.3,3.6,...,23,98,15,733,31,37,15625000,73.0,9,MIL
5,Ersan İlyasova,32,56,8,16.0,2.4,5.2,0.472,0.8,2.2,...,5,31,17,117,7,13,7000000,81.0,11,MIL
6,Frank Mason,25,6,0,8.8,0.8,3.2,0.263,0.3,1.7,...,0,2,0,29,0,3,NaN,71.0,2,MIL
7,George Hill,33,52,0,21.2,3.4,6.5,0.530,1.4,2.9,...,4,24,16,361,4,20,9133907,75.0,11,MIL
8,Giannis Antetokounmpo,25,57,57,30.9,10.9,20.0,0.547,1.5,4.8,...,55,276,10,836,86,64,25842697,83.0,6,MIL
9,Khris Middleton,28,55,52,30.1,7.7,15.5,0.499,2.4,5.8,...,19,73,10,519,15,33,30603448,79.0,7,MIL


In [40]:
#fix percent as position
for column in ['PercasPG', 'PercasSG', 'PercasSF', 'PercasPF', 'PercasC']:    
    all_individuals[column].fillna('0%', inplace = True) 

for idx, x in enumerate(all_individuals.PercasPG):
        all_individuals.loc[idx, 'PercasPG'] = x[:-1]
        all_individuals.loc[idx, 'PercasSG'] = all_individuals.PercasSG.iloc[idx][:-1]
        all_individuals.loc[idx, 'PercasSF'] = all_individuals.PercasSF.iloc[idx][:-1]
        all_individuals.loc[idx, 'PercasPF'] = all_individuals.PercasPF.iloc[idx][:-1]
        all_individuals.loc[idx, 'PercasC'] = all_individuals.PercasC.iloc[idx][:-1]
    
all_individuals.PercasPG = all_individuals.PercasPG.astype('float64')
all_individuals.PercasSG = all_individuals.PercasSG.astype('float64')
all_individuals.PercasSF = all_individuals.PercasSF.astype('float64')
all_individuals.PercasPF = all_individuals.PercasPF.astype('float64')
all_individuals.PercasC = all_individuals.PercasC.astype('float64')

all_individuals.PercasPG = all_individuals.PercasPG/100
all_individuals.PercasSG = all_individuals.PercasSG/100
all_individuals.PercasSF = all_individuals.PercasSF/100
all_individuals.PercasPF = all_individuals.PercasPF/100
all_individuals.PercasC = all_individuals.PercasC/100

In [41]:
#add other stats
for idx, value in enumerate(all_individuals.Name):
    val = (all_individuals['PTS/G'].iloc[idx])*(all_individuals['TRB/G'].iloc[idx])*(all_individuals['AST/G'].iloc[idx])
    all_individuals.loc[idx, 'VI'] = np.sign(val)*(np.abs(val)**(1/3))

    percpos = max(all_individuals['PercasPG'].iloc[idx], all_individuals['PercasSG'].iloc[idx], all_individuals['PercasSF'].iloc[idx],
                  all_individuals['PercasPF'].iloc[idx], all_individuals['PercasC'].iloc[idx])
    all_individuals.loc[idx, 'PAWS'] = ((all_individuals['WS/48'].iloc[idx])/48)*((all_individuals['MP/G'].iloc[idx])*percpos)

    all_individuals.loc[idx, 'NSPTotal'] = ((all_individuals['FGATotal'].iloc[idx])-(all_individuals['FGMadeTotal'].iloc[idx])) + 0.4*(all_individuals['FTATotal'].iloc[idx]) + (all_individuals['TOVTotal'].iloc[idx])
    all_individuals.loc[idx, 'NSP/G'] = (all_individuals['NSPTotal'].iloc[idx])/(all_individuals['G'].iloc[idx])

    all_individuals.loc[idx, 'SL'] = 27 - (0.75*(all_individuals['Age'].iloc[idx]))

    credits=(all_individuals['PTSTotal'].iloc[idx]+all_individuals['TRBTotal'].iloc[idx]+all_individuals['ASTTotal'].iloc[idx]+all_individuals['STLTotal'].iloc[idx]+all_individuals['BLKTotal'].iloc[idx]-(all_individuals['FGATotal'].iloc[idx]-all_individuals['FGMadeTotal'].iloc[idx])-(all_individuals['FTATotal'].iloc[idx]-all_individuals['FTMadeTotal'].iloc[idx])-all_individuals['TOVTotal'].iloc[idx])
    all_individuals.loc[idx, 'AV'] = (np.sign(credits)*(np.abs(credits)**(3/4)))/21

In [44]:
all_individuals

,Name,Age,G,GS,MP/G,FGMade/G,FGA/G,FGPerc,ThreeMade/G,ThreeA/G,...,Salary,Ht,Exp,Team,VI,PAWS,NSPTotal,NSP/G,SL,AV
0,Brook Lopez,31,61,60,26.6,4.0,9.4,0.427,1.4,4.7,...,12093024,84.0,11,MIL,4.294458,0.075921,438.4,7.186885,3.75,7.270233
1,D.J. Wilson,23,31,0,9.0,1.3,3.3,0.396,0.5,1.8,...,2961120,82.0,2,MIL,1.701728,0.006497,79.6,2.567742,9.75,1.704874
2,Donte DiVincenzo,23,59,22,23.1,3.5,7.7,0.462,1.3,3.7,...,2905800,76.0,1,MIL,4.731701,0.048664,346.2,5.867797,9.75,6.783603
3,Dragan Bender,22,7,0,13.0,1.4,3.0,0.476,0.6,1.3,...,1678854,NaN,NaN,MIL,2.407212,0.027706,17.2,2.457143,10.50,0.799617
4,Eric Bledsoe,30,56,56,27.2,5.7,11.8,0.482,1.3,3.6,...,15625000,73.0,9,MIL,7.259234,0.095200,560.2,10.003571,4.50,8.372552
5,Ersan İlyasova,32,56,8,16.0,2.4,5.2,0.472,0.8,2.2,...,7000000,81.0,11,MIL,2.987205,0.051600,216.0,3.857143,3.00,5.356493
6,Frank Mason,25,6,0,8.8,0.8,3.2,0.263,0.3,1.7,...,NaN,71.0,2,MIL,1.752459,-0.011000,21.6,3.600000,8.25,0.326015
7,George Hill,33,52,0,21.2,3.4,6.5,0.530,1.4,2.9,...,9133907,75.0,11,MIL,4.371161,0.084535,240.2,4.619231,2.25,5.973797
8,Giannis Antetokounmpo,25,57,57,30.9,10.9,20.0,0.547,1.5,4.8,...,25842697,83.0,6,MIL,13.298830,0.139784,953.0,16.719298,8.25,14.161234
9,Khris Middleton,28,55,52,30.1,7.7,15.5,0.499,2.4,5.8,...,30603448,79.0,7,MIL,8.124924,0.106015,617.4,11.225455,6.00,9.866169


In [59]:
option = webdriver.ChromeOptions()
option.add_argument(" — incognito")
driver = webdriver.Chrome(executable_path='/Users/Anubhav/Desktop/Flatiron School/QCstats/chromedriver', options=option)
driver.get(urls[0])
WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.ID, "all_roster")))

<selenium.webdriver.remote.webelement.WebElement (session="8388c0ba9926d09ac5e458039c4f3994", element="7c40e5f1-eb50-40b0-9036-fd21e8e26456")>

In [120]:
teamstats = pd.read_html(driver.find_element_by_css_selector('table[id="team_and_opponent"]').get_attribute('outerHTML'))[0]

In [121]:
teamstats = pd.read_html(driver.find_element_by_css_selector('table[id="team_and_opponent"]').get_attribute('outerHTML'))[0]

yerp = teamstats[teamstats.index == 0].drop('Unnamed: 0', axis=1)
yerp.reset_index(drop=True, inplace=True)
yerp1 = teamstats[teamstats.index == 1].drop(['Unnamed: 0', 'G', 'FG%', '3P%', '2P%', 'FT%'], axis=1)
yerp1.reset_index(drop=True, inplace=True)
yerp2 = teamstats[teamstats.index == 4].drop('Unnamed: 0', axis=1)
yerp2.reset_index(drop=True, inplace=True)
yerp3 = teamstats[teamstats.index == 5].drop(['Unnamed: 0', 'G', 'FG%', '3P%', '2P%', 'FT%'], axis=1)
yerp3.reset_index(drop=True, inplace=True)
yerp4 = teamstats[teamstats.index == 3].drop(['Unnamed: 0', 'G'], axis=1)
yerp4.reset_index(drop=True, inplace=True)
yerp5 = teamstats[teamstats.index == 4].drop(['Unnamed: 0', 'G'], axis=1)
yerp5.reset_index(drop=True, inplace=True)

yerp.columns = ['G', 'MPTotal', 'FGMadeTotal', 'FGATotal', 'FGPerc', 'ThreeMadeTotal', 'ThreeATotal',
 'ThreePerc', 'TwoMadeTotal', 'TwoATotal', 'TwoPerc', 'FTMadeTotal', 'FTATotal',
 'FTPerc', 'ORBTotal', 'DRBTotal', 'TRBTotal', 'ASTTotal', 'STLTotal','BLKTotal',
 'TOVTotal', 'PFTotal', 'PTSTotal']

yerp1.columns = ['MP/G', 'FGMade/G', 'FGA/G', 'ThreeMade/G', 'ThreeA/G',
                 'TwoMade/G', 'TwoA/G', 'FTMade/G', 'FTA/G', 'ORB/G',
                 'DRB/G', 'TRB/G', 'AST/G', 'STL/G','BLK/G', 'TOV/G', 'PF/G', 'PTS/G']

yerp2.columns = ['OppG', 'OppMPTotal', 'OppFGMadeTotal', 'OppFGATotal', 'OppFGPerc',
                 'OppThreeMadeTotal', 'OppThreeATotal', 'OppThreePerc', 'OppTwoMadeTotal',
                 'OppTwoATotal', 'OppTwoPerc', 'OppFTMadeTotal', 'OppFTATotal', 'OppFTPerc',
                 'OppORBTotal', 'OppDRBTotal', 'OppTRBTotal', 'OppASTTotal', 'OppSTLTotal',
                 'OppBLKTotal', 'OppTOVTotal', 'OppPFTotal', 'OppPTSTotal']

yerp3.columns = ['OppMP/G', 'OppFGMade/G', 'OppFGA/G', 'OppThreeMade/G', 'OppThreeA/G',
                 'OppTwoMade/G', 'OppTwoA/G', 'OppFTMade/G', 'OppFTA/G', 'OppORB/G',
                 'OppDRB/G', 'OppTRB/G', 'OppAST/G', 'OppSTL/G','OppBLK/G', 'OppTOV/G',
                 'OppPF/G', 'OppPTS/G']

yerp4.columns = ['MPDiff', 'FGMadeDiff', 'FGADiff', 'FGDiff', 'ThreeMadeDiff', 'ThreeADiff',
                 'ThreePercDiff', 'TwoMadeDiff', 'TwoADiff', 'TwoDiff', 'FTMadeDiff', 'FTADiff',
                 'FTPercDiff', 'ORBDiff', 'DRBDiff', 'TRBDiff', 'ASTDiff', 'STLDiff','BLKDiff',
                 'TOVDiff', 'PFDiff', 'PTSDiff']

yerp5.columns = ['OppMPDiff', 'OppFGMadeDiff', 'OppFGADiff', 'OppFGDiff', 'OppThreeMadeDiff', 'OppThreeADiff',
                 'OppThreePercDiff', 'OppTwoMadeDiff', 'OppTwoADiff', 'OppTwoDiff', 'OppFTMadeDiff', 'OppFTADiff',
                 'OppFTPercDiff', 'OppORBDiff', 'OppDRBDiff', 'OppTRBDiff', 'OppASTDiff', 'OppSTLDiff','OppBLKDiff',
                 'OppTOVDiff', 'OppPFDiff', 'OppPTSDiff']

misc = pd.read_html(driver.find_element_by_css_selector('table[id="team_misc"]').get_attribute('outerHTML'))[0]

misc.columns = ['Label', 'W', 'L', 'PW', 'PL', 'MOV', 'SOS',
                'SRS', 'ORtg', 'DRtg', 'Pace', 'FTr', '3PAr',
                'OeFGPerc', 'OTOVPerc', 'ORBPerc', 'OFT/FGA', 'DeFGPerc',
                'DTOVPerc', 'DRBPerc', 'DFT/FGA', 'Arena', 'Attendance']

yerp6 = misc[misc.index == 0].drop(['Label', 'Arena', 'Attendance'], axis=1)
yerp6.reset_index(drop=True, inplace=True)

teamstatsrow = pd.concat([yerp, yerp1, yerp2, yerp3, yerp4, yerp5, yerp6], axis=1)

In [19]:
teamstats = pd.read_html(driver.find_element_by_css_selector('table[id="team_and_opponent"]').get_attribute('outerHTML'))[0]

yerp = teamstats[teamstats.index == 0].drop('Unnamed: 0', axis=1)
yerp.reset_index(drop=True, inplace=True)
yerp1 = teamstats[teamstats.index == 1].drop(['Unnamed: 0', 'G', 'FG.1', '3P.1', '2P.1', 'FT.1'], axis=1)
yerp1.reset_index(drop=True, inplace=True)
yerp2 = teamstats[teamstats.index == 4].drop('Unnamed: 0', axis=1)
yerp2.reset_index(drop=True, inplace=True)
yerp3 = teamstats[teamstats.index == 5].drop(['Unnamed: 0', 'G', 'FG.1', '3P.1', '2P.1', 'FT.1'], axis=1)
yerp3.reset_index(drop=True, inplace=True)
yerp4 = teamstats[teamstats.index == 3].drop(['Unnamed: 0', 'G'], axis=1)
yerp4.reset_index(drop=True, inplace=True)
yerp5 = teamstats[teamstats.index == 4].drop(['Unnamed: 0', 'G'], axis=1)
yerp5.reset_index(drop=True, inplace=True)

In [21]:
teamstats

,Unnamed: 0,G,MP,FG,FGA,FG.1,3P,3PA,3P.1,2P,...,FT.1,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,Team,65.0,15650,2830,5928,0.477,893,2510,0.356,1937,...,0.742,617,2745,3362,1685,480,390,969,1247,7712
1,Team/G,NaN,240.8,43.5,91.2,0.477,13.7,38.6,0.356,29.8,...,0.742,9.5,42.2,51.7,25.9,7.4,6.0,14.9,19.2,118.6
2,Lg Rank,NaN,15,1,3,2.000,5,4,16.000,8,...,27.000,25,1,1,7,20,3,19,6,1
3,Year/Year,NaN,-0.2%,0.4%,0.1%,0.002,2.0%,1.0%,0.003,-0.3%,...,-0.031,2.1%,4.4%,4.0%,-0.5%,-1.5%,1.2%,7.5%,-2.2%,0.4%
4,Opponent,65.0,15650,2518,6097,0.413,892,2509,0.356,1626,...,0.784,613,2358,2971,1547,481,297,929,1386,6978
5,Opponent/G,NaN,240.8,38.7,93.8,0.413,13.7,38.6,0.356,25.0,...,0.784,9.4,36.3,45.7,23.8,7.4,4.6,14.3,21.3,107.4
6,Lg Rank,NaN,15,3,30,1.000,30,30,18.000,2,...,26.000,7,27,21,11,12,11,18,8,5
7,Year/Year,NaN,-0.2%,-4.4%,0.3%,-0.020,4.9%,6.4%,-0.005,-8.8%,...,0.051,-4.9%,2.5%,0.9%,-3.9%,8.9%,-5.1%,6.9%,5.8%,-1.7%


In [14]:
yerp1 = teamstats[teamstats.index == 1]

In [17]:
teamstats.columns

Index(['Unnamed: 0', 'G', 'MP', 'FG', 'FGA', 'FG.1', '3P', '3PA', '3P.1', '2P',
       '2PA', '2P.1', 'FT', 'FTA', 'FT.1', 'ORB', 'DRB', 'TRB', 'AST', 'STL',
       'BLK', 'TOV', 'PF', 'PTS'],
      dtype='object')